In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path
import json
import numpy as np
from yolov7.utils.data_preparation import copy_split
from tools.evaluation import coco2kitti
import shutil

## Split Training and Test Data

In [3]:
# Paths to image and label data (for loading the KITTI dataset within Kaggle)
base_dir = Path.cwd().parent.parent / "input"
print(f"Base Directory : {base_dir}")
img_path = base_dir / "images" / "data_object_image_2" / "training" / "image_2"
label_path = base_dir / "labels"

# Loading the classes present in the dataset
with open(base_dir / "classes.json", "r") as f:
    classes = json.load(f)
print(f"Classes : {classes}")

# Sorting images and labels to ensure alignment
ims = sorted(img_path.glob("*"))
labels = sorted(label_path.glob("*"))
pairs = list(zip(ims, labels))

# Dataset shuffle for randomized train/test split
seed = 42  # For reproducibility
random_state = np.random.RandomState(seed)
random_state.shuffle(pairs)

# Calculating the test size (10%)
test_size = int(0.1 * len(pairs))
splits = {}

# Creating 3 distinct splits
for i in range(3):
    # Select the test set for this split
    test_set = pairs[i * test_size : (i + 1) * test_size]
    # Select the training set (remaining data)
    train_set = pairs[: i * test_size] + pairs[(i + 1) * test_size :]
    splits[f"split{i + 1}"] = {"train": train_set, "test": test_set}

# Verifying sizes of each split
for key, value in splits.items():
    train_size = len(value["train"])
    test_size = len(value["test"])
    print(f"{key} - Train Size: {train_size}, Test Size: {test_size}")
    assert train_size + test_size == len(
        pairs
    ), "Train and test sizes do not add up to total pairs"

# Verifying distinct test sets
for i in range(3):
    for j in range(i + 1, 3):
        assert not set(splits[f"split{i + 1}"]["test"]).intersection(
            set(splits[f"split{j + 1}"]["test"])
        ), f"Test sets for split{i + 1} and split{j + 1} overlap"


Base Directory : D:\dev-workspace\IRO\input
Classes : {'Car': 0, 'Pedestrian': 1, 'Van': 2, 'Cyclist': 3, 'Truck': 4, 'Misc': 5, 'Tram': 6, 'Person_sitting': 7}
split1 - Train Size: 6733, Test Size: 748
split2 - Train Size: 6733, Test Size: 748
split3 - Train Size: 6733, Test Size: 748


## Prepare Data

Prepare the necessary data for the mAP evaluation in the following folders:

```bash
./
├── mAP/input/                        # Necessary files for mAP evaluation
│   ├── detection-results/            # Inference results on KITTI test data
│   ├── ground-truth                  # Ground truth labels
│   ├── images                         # Ground truth images
│   └── images-optional               # (optional) images for animation
...
```

In [4]:
eval_path = Path('../mAP/input').resolve()

### Copy Test Dataset

In [24]:
if eval_path.exists():
    shutil.rmtree(eval_path / 'ground-truth')
if eval_path.exists():
    shutil.rmtree(eval_path / 'images')

(eval_path / 'ground-truth').mkdir(exist_ok=True)
(eval_path / 'images').mkdir(exist_ok=True)

# (optional) When using animation
if eval_path.exists():
    shutil.rmtree(eval_path / 'images-optional')
(eval_path / 'images-optional').mkdir(exist_ok=True)

In [25]:
# Get test data split index and path
# test_split = splits['split1']['test']
test_split = splits['split2']['test']
# test_split = splits['split3']['test']

# Copy test dataset to a designated folder
copy_split(test_split, eval_path, separate_labels=True, label_folder_name='ground-truth', img_folder_name='images')

# (optional) Copy images to images-optional when using animation
shutil.copytree(eval_path / 'images', eval_path / 'images-optional', dirs_exist_ok=True)

100%|██████████| 748/748 [00:01<00:00, 640.68it/s]


WindowsPath('D:/dev-workspace/IRO/evaluation/mAP/input/images-optional')

In [26]:
# Only for Fine-tuned YOLOv7 vs. COCO Pretrained YOLOv7 evaluation
# coco2kitti(eval_path / 'ground-truth', convert_type="groundtruth")

### Copy YOLOv7 Detection Results

In [27]:
if eval_path.exists():
    shutil.rmtree(eval_path / 'detection-results')

(eval_path / 'detection-results').mkdir(exist_ok=True)

In [28]:
detection_path = Path('./detections')

# Only for Fine-tuned YOLOv7 vs. COCO Pretrained YOLOv7 evaluation
# coco2kitti(detection_path, convert_type="detection")

eval_detection_path = eval_path / 'detection-results'

shutil.copytree(detection_path, eval_detection_path, dirs_exist_ok=True)

WindowsPath('D:/dev-workspace/IRO/evaluation/mAP/input/detection-results')

### Convert Label Formats

In [29]:
if (eval_detection_path / 'backup').exists():
    shutil.rmtree(eval_detection_path / 'backup')
if (eval_path / 'ground-truth' / 'backup').exists():
    shutil.rmtree(eval_path / 'ground-truth' / 'backup')

In [30]:
# Convert detection labels
!python ../mAP/scripts/extra/convert_kitti_yolov7.py            

# Convert ground truth labels
!python ../mAP/scripts/extra/convert_kitti_yolov7.py  -gt         

Conversion completed!
Conversion completed!


## Evaluate

In [31]:
if (Path('../mAP/output')).exists():
    shutil.rmtree(Path('../mAP/output'))

In [32]:
!python ../mAP/main.py

67.40% = Car AP 
49.36% = Pedestrian AP 
22.63% = Tram AP 
9.25% = Truck AP 
mAP = 37.16%
Figure(640x480)


[ WARN:0@431.270] global loadsave.cpp:241 cv::findDecoder imread_('output/images/001315.png'): can't open/read file: check file path/integrity
[ WARN:0@432.431] global loadsave.cpp:241 cv::findDecoder imread_('output/images/001554.png'): can't open/read file: check file path/integrity
[ WARN:0@434.457] global loadsave.cpp:241 cv::findDecoder imread_('output/images/001894.png'): can't open/read file: check file path/integrity
[ WARN:0@436.249] global loadsave.cpp:241 cv::findDecoder imread_('output/images/002417.png'): can't open/read file: check file path/integrity
[ WARN:0@436.531] global loadsave.cpp:241 cv::findDecoder imread_('output/images/002498.png'): can't open/read file: check file path/integrity
[ WARN:0@441.331] global loadsave.cpp:241 cv::findDecoder imread_('output/images/003918.png'): can't open/read file: check file path/integrity
[ WARN:0@442.841] global loadsave.cpp:241 cv::findDecoder imread_('output/images/004354.png'): can't open/read file: check file path/integrity